In [1]:
# Load the different graphs

# Compare the pairwise clusters

# What occupation belong the same clusters ? 

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("../database.db")
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('region_gini_interets',), ('df_cleaned_occupations',), ('individual_id_cleaned_occupations',), ('temporal_data',), ('optimization',), ('region_optimized_partition',), ('temporal_data_clean',), ('individual_score',), ('optimal_partition',), ('partition_before_1500',), ('partition_global',), ('partition_non_europe',), ('partition_top_individuals',), ('optimal_partition_weighted',)]


In [4]:
import pandas as pd

In [5]:
df_partition_before_1500 = pd.read_sql("SELECT * FROM partition_before_1500", conn)
df_partition_before_1500 = df_partition_before_1500.sort_values('node', ascending=False).reset_index(drop=True)

df_partition_non_europe = pd.read_sql("SELECT * FROM partition_non_europe", conn)
df_partition_non_europe = df_partition_non_europe.sort_values('node', ascending=False).reset_index(drop=True)

df_partition_top_individuals = pd.read_sql("SELECT * FROM partition_top_individuals", conn)
df_partition_top_individuals = df_partition_top_individuals.sort_values('node', ascending=False).reset_index(drop=True)

df_partition_global = pd.read_sql("SELECT * FROM partition_global", conn)
df_partition_global = df_partition_global.sort_values('node', ascending=False).reset_index(drop=True)

df_optimal_partition_weighted =  pd.read_sql("SELECT * FROM optimal_partition_weighted ", conn)
df_optimal_partition_weighted = df_optimal_partition_weighted.sort_values('node', ascending=False).reset_index(drop=True)

In [6]:
from sklearn.metrics import adjusted_rand_score
import pandas as pd
pd.options.mode.chained_assignment = None


# Assuming you've loaded your datasets as described in the question

datasets = {
    'Before 1500': df_partition_before_1500,
    'Non-Europe': df_partition_non_europe,
    'Top Individuals': df_partition_top_individuals,
    'Global': df_partition_global,
    'Weighted': df_optimal_partition_weighted
}

# Create an empty DataFrame to store the ARI values
ari_df = pd.DataFrame(columns=['Dataset 1', 'Dataset 2', 'ARI'])


# Iterate over pairs of datasets and compute the ARI
for dataset1_name, dataset1 in datasets.items():
    for dataset2_name, dataset2 in datasets.items():
        if dataset1_name != dataset2_name:
            # Get the common elements based on the 'node' column
            common_elements = set(dataset1['node']).intersection(dataset2['node'])
            
            # Filter the datasets to include only the common elements
            filtered_dataset1 = dataset1[dataset1['node'].isin(common_elements)]
            filtered_dataset2 = dataset2[dataset2['node'].isin(common_elements)]
            
            ari = adjusted_rand_score(filtered_dataset1['community'], filtered_dataset2['community'])
            ari_df = ari_df.append({'Dataset 1': dataset1_name, 'Dataset 2': dataset2_name, 'ARI': ari}, ignore_index=True)

# Print or analyze the ARI values as needed

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_90205/3431362841.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ari_df = ari_df.append({'Dataset 1': dataset1_name, 'Dataset 2': dataset2_name, 'ARI': ari}, ignore_index=True)
/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_90205/3431362841.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ari_df = ari_df.append({'Dataset 1': dataset1_name, 'Dataset 2': dataset2_name, 'ARI': ari}, ignore_index=True)
/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_90205/3431362841.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ari_df = ari_df.append({'Dataset 1': dataset1_name, 'Dataset 2': dataset2_name, 'ARI': ari}, ignore_index=True)
/var/folde

In [15]:
ari_df.ARI.mean()

0.6978431725921397

In [14]:
matrix_df = ari_df.pivot_table(index='Dataset 1', columns='Dataset 2', values='ARI', aggfunc='mean')
matrix_df = matrix_df.fillna(1)
matrix_df.to_clipboard()
matrix_df

Dataset 2,Before 1500,Global,Non-Europe,Top Individuals,Weighted
Dataset 1,,,,,
Before 1500,1.000000,0.563659,0.554548,0.757737,0.650404
Global,0.563659,1.000000,0.690539,0.704312,0.895031
Non-Europe,0.554548,0.690539,1.000000,0.593429,0.783887
Top Individuals,0.757737,0.704312,0.593429,1.000000,0.784885
Weighted,0.650404,0.895031,0.783887,0.784885,1.000000


#### Find the pair that always tsay together

In [9]:
# Sample datasets
dataset1 = datasets['Global']
dataset1.columns = ['node', 'com1']

dataset2 = datasets['Before 1500']
dataset2.columns = ['node', 'com2']

dataset3 = datasets['Weighted']
dataset3.columns = ['node', 'com3']

dataset4 = datasets['Top Individuals']
dataset4.columns = ['node', 'com4']

dataset5 = datasets['Non-Europe']
dataset5.columns = ['node', 'com5']

df = pd.merge(dataset1, dataset2, on = 'node')
df = pd.merge(df, dataset3, on = 'node')
df = pd.merge(df, dataset4, on = 'node')
df = pd.merge(df, dataset5, on = 'node')

final = []
for occ_1 in list(df.node):
    for occ_2 in list(df.node):
        if occ_1 == occ_2:
            pass
        else:
            res = df[df['node'].isin([occ_1, occ_2])]
            
            count_elements = []
            for col in df.columns[1:]:
                count_element = len(set(list(res[col])))
                count_elements.append(count_element)
                
            if count_elements == [1,1,1,1,1]:
                final.append({'occ_1':occ_1, 'occ_2':occ_2})

In [10]:
test =pd.DataFrame(final)
len(set(test.occ_1))

20